# Preprocessing Enron and Nazario Datasets


In [ ]:
import os
import sys
import pandas as pd

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data_preprocessing.maildir_to_mbox import maildir_to_mbox
from src.data_preprocessing.dataset_to_dataframe import dataset_to_dataframe

# Specify the output path for mbox creation
legit_mbox_path = "../../data/legit.mbox"
phishing_dir_path = "../../data/raw/phishing/"
mixed_dataset_path = "../../data/mixed_dataset.csv"

## Legit

We will start the data preprocessing by converting Enron mail set from maildir format to mbox format.


In [ ]:
maildir_to_mbox(legit_mbox_path, mail_dir_pattern="../../data/raw/legit/*/inbox")

Then, mbox formatted mails will be inserted in a dataframe


In [ ]:
df_legit = dataset_to_dataframe(legit_mbox_path)

We noticed that some emails didn’t have subject fields. For the uniformity of our dataset, we have decided to remove those rows


In [ ]:
empty_subject_rows = df_legit[
    df_legit["Subject"].apply(lambda x: str(x).strip()) == ""
]  # Select rows with empty subjects


display(empty_subject_rows.head())


empty_subject_rows.info()

In [ ]:
df_legit = df_legit[~df_legit["Subject"].astype(str).str.strip().eq("")]
df_legit.info()

Lastly, duplicate emails will be removed. As seen from the previous info, there has been a lot of duplicate emails deleted.


In [ ]:
df_legit.drop_duplicates(subset="Body", keep="first", inplace=True)
df_legit.info()

## Phishing

The nazario dataset contains different .mbox files. So, they are converted to individual dataframes and then are concatanated to form a single dataframe.


In [ ]:
phish_path_list = [_ for _ in os.listdir(phishing_dir_path)]

# Initialize an empty list to store individual dataframes
dataframes = []

# Iterate through each path and convert it to a dataframe
for path in phish_path_list:
    path_to_file = os.path.join(phishing_dir_path, path)
    if os.path.isfile(path_to_file):  # Check if it's a file
        print(path)
        df = dataset_to_dataframe(path_to_file)
        dataframes.append(df)
        print()

# Concatenate all dataframes into a final dataframe
df_phish = pd.concat(dataframes, ignore_index=True)

In [ ]:
df_phish.info()

As seen in the info, there are some null values inside our dataframe. We can check the rows that contain a null value by:


In [ ]:
print("Null valued lines in the phishing df")
display(df_phish[df_phish.isnull().any(axis=1)])

As seen, there are a few rows with None values inside the phishing df. These rows will be removed from the dataframe.


In [ ]:
df_phish = df_phish.dropna()

Also, there are some mails with empty body and subject fields in this dataset. They will also be removed.


In [ ]:
# Filtering empty body fields
empty_body_rows = df_phish[df_phish["Body"].astype(str).str.strip().eq("")]
display(empty_body_rows.head())
df_phish = df_phish[~df_phish["Body"].astype(str).str.strip().eq("")]

# Filtering empty subject fields
empty_subject_rows = df_phish[df_phish["Subject"].astype(str).str.strip().eq("")]
display(empty_subject_rows)
df_phish = df_phish[~df_phish["Subject"].astype(str).str.strip().eq("")]

Then, we will drop the duplicate rows.


In [ ]:
df_phish.info()
print()
df_phish.drop_duplicates(subset="Body", keep="first", inplace=True)
df_phish.info()

# Forming the datasets


In this part, we will form a 1:4 dataset using preprocessed data. The formed mixed dataset will have 2000 phishing and 8000 legit mails. A new column "isPhishing" will be added to label each mail.

"seed" value can be used to obtain the same result or different results when forming the dataframe.


In [ ]:
seed = 4444

max_sample_size = min(2000, len(df_phish))

# Sample the dataframes randomly
sampled_phish = df_phish.sample(n=max_sample_size, random_state=seed)
sampled_legit = df_legit.sample(n=max_sample_size * 4, random_state=seed)

# Classify the dataframes as phishing or not phishing
sampled_legit["isPhishing"] = 0
sampled_phish["isPhishing"] = 1

mixed_dataset = pd.concat([sampled_phish, sampled_legit])

# Shuffle the mixed dataset
mixed_dataset = mixed_dataset.sample(frac=1, random_state=seed).reset_index(drop=True)
mixed_dataset.to_csv(mixed_dataset_path, index=False, errors="replace")

mixed_dataset.shape

In [ ]:
mixed_dataset.head()